In [1]:
import ROOT as rt
import csv
import re
import sys
import collections
from collections import OrderedDict
import uproot
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
from numpy import linalg as LA
from ROOT import TLorentzVector
from array import array

import math

# sys.path.append('/home/cms/Desktop/LLP/cms_lpc_llp/jet_timing_studies/python/lib')
# sys.path.append('/nfshome/christiw/llp/cms_lpc_llp/jet_timing_studies/python/lib')
sys.path.append('/nfshome/christiw/llp/delayed_jet_analyzer/lib/')
# sys.path.append('/nfshome/christiw/llp/cms_lpc_llp/llp_analyzer/python/lib')

from histo_utilities import create_TH1D, create_TH2D, create_TGraph,std_color_list
     
import CMS_lumi, tdrstyle 
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Simulation Preliminary"

# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.16/00
3.6.8 (default, Apr 25 2019, 21:02:35) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-36)]


In [6]:
directory = '/mnt/hadoop/store/group/phys_exotica/jmao/jet_timing_studies/samples_plus/'
out_dir = '/nfshome/jmao/cms-llp/plots/20190719/'
sig_string = 'signal/bbbb/ggh/'
sig_name_string1 = 'ntuple_RunIISummer16_'
sig_name_string2 = '.root'
bkg_string = 'qcd/'
bkg_name_string1 = 'ntuple_RunIIFall17_'
bkg_name_string2 = '_TuneCP5_13TeV_pythia8.root'


# labels = {'QCD_Pt_15to30',
labels = {
          'bbbb_ggh_mh125_mx50_pl10000',
          'bbbb_ggh_mh125_mx50_pl1000',
          'bbbb_ggh_mh2000_mx975_pl1000'
          }

texs = {}

texs['QCD_Pt_15to30'] = 'QCD_Pt_15to30'
texs['bbbb_ggh_mh125_mx50_pl10000'] = 'm_{h} = 125 GeV, m_{x} = 50 GeV, c#tau = 10m'
texs['bbbb_ggh_mh125_mx50_pl1000'] = 'm_{h} = 125 GeV, m_{x} = 50 GeV, c#tau = 1m'
texs['bbbb_ggh_mh2000_mx975_pl1000'] = 'm_{h} = 2000 GeV, m_{x} = 975 GeV, c#tau = 1m'

cols = {}

cols['QCD_Pt_15to30'] = 6
cols['bbbb_ggh_mh125_mx50_pl10000'] =  1
cols['bbbb_ggh_mh125_mx50_pl1000'] =  2
cols['bbbb_ggh_mh2000_mx975_pl1000'] = 4 


MODEL = 'pp #rightarrow XX, X #rightarrow bb, X #rightarrow bb, ggH'

fpath = OrderedDict()
tree = OrderedDict()
nevts = OrderedDict()

 

for label in labels:
    print(label)
    if 'bb' in label:
        fpath[label] = directory + sig_string + sig_name_string1 + label + sig_name_string2
    else:
        fpath[label] = directory + bkg_string + bkg_name_string1 + label + bkg_name_string2

    print (label, fpath[label])

    print ('File ready')


    root_dir = uproot.open(fpath[label]) 
    print ('Open ready')

    tree[label] = root_dir['ntuples/llp']
    print ('Tree ready')

    nevts[label] = root_dir['ntuples/NEvents'][1]
    print ('NEvents = ', nevts[label])
    print ('NEvents ready')


bbbb_ggh_mh125_mx50_pl10000
bbbb_ggh_mh125_mx50_pl10000 /mnt/hadoop/store/group/phys_exotica/jmao/jet_timing_studies/samples_plus/signal/bbbb/ggh/ntuple_RunIISummer16_bbbb_ggh_mh125_mx50_pl10000.root
File ready
Open ready
Tree ready
NEvents =  50973.0
NEvents ready
bbbb_ggh_mh125_mx50_pl1000
bbbb_ggh_mh125_mx50_pl1000 /mnt/hadoop/store/group/phys_exotica/jmao/jet_timing_studies/samples_plus/signal/bbbb/ggh/ntuple_RunIISummer16_bbbb_ggh_mh125_mx50_pl1000.root
File ready
Open ready
Tree ready
NEvents =  50918.0
NEvents ready
bbbb_ggh_mh2000_mx975_pl1000
bbbb_ggh_mh2000_mx975_pl1000 /mnt/hadoop/store/group/phys_exotica/jmao/jet_timing_studies/samples_plus/signal/bbbb/ggh/ntuple_RunIISummer16_bbbb_ggh_mh2000_mx975_pl1000.root
File ready
Open ready
Tree ready
NEvents =  17362.0
NEvents ready


In [5]:
#sort lists based on list1 and require list1 to be valid
def sort_valid_lists(list1, list2): 
    list3 = []
    for index, (ele1,ele2) in enumerate(zip(list1,list2)): 
#         print(index,ele1,ele2)
        if not math.isnan(ele1):
#             print("ele1 is not nan")
            list3.append(tuple((ele1,ele2)))
    list3.sort(key=lambda x: x[0])
    return list3 